In [1]:
#pip install tensorflow

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [3]:
# Set the path for image directory
image_dir = Path(r'C:\Users\bowma\Programming\Datasets\images')

# Get all image file paths
filepaths = list(image_dir.glob(r'**/*.jpg'))

# Extract labels by extracting the parent folder name
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

# Create a DataFrame with filepaths and labels
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
images = pd.concat([filepaths, labels], axis=1)

# Create a subset with 100 samples per category for balancing
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))

# Combine the samples and shuffle
image_df = pd.concat(category_samples, axis=0).sample(
frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
# Check the class distribution
image_df['Label'].value_counts()

Label
paella                  100
red_velvet_cake         100
baby_back_ribs          100
shrimp_and_grits        100
cup_cakes               100
                       ... 
takoyaki                100
pad_thai                100
fried_rice              100
strawberry_shortcake    100
apple_pie               100
Name: count, Length: 101, dtype: int64

In [5]:
# Split the dataset into training and test sets (70% train, 30% test)
train_df, test_df = train_test_split(
    image_df, train_size=0.7, shuffle=True, random_state=42)

In [6]:
# Image data generators for preprocessing and augmentation
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2  # 20% for validation from the training set
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)


In [7]:

# Flow training data from the dataframe with batching and augmentation
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),  # Resize to 224x224 (MobileNetV2 input size)
    color_mode='rgb',
    class_mode='categorical',  # Multi-class classification
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

# Flow validation data from the dataframe (for model evaluation during training)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

# Flow test data (no shuffling, used only for evaluation)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 5656 validated image filenames belonging to 101 classes.
Found 1414 validated image filenames belonging to 101 classes.
Found 3030 validated image filenames belonging to 101 classes.


In [8]:
#Load the pretrained MobileNetV2 model without the top layer (to add custom layers on top)
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,  # Exclude the fully connected layer at the top
    weights='imagenet',  # Load pre-trained weights from ImageNet
    pooling='avg'  # Global average pooling (average pooling of the feature maps)
)
pretrained_model.trainable = False  # Freeze the pretrained layers


In [20]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(101, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,451,493 (9.35 MB)

 Trainable params: 193,509 (755.89 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

None


In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True)])

C:\Users\bowma\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 108s 509ms/step - accuracy: 0.0718 - loss: 4.3184 - val_accuracy: 0.2801 - val_loss: 3.0009
Epoch 2/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 89s 503ms/step - accuracy: 0.3776 - loss: 2.5013 - val_accuracy: 0.3833 - val_loss: 2.4953
Epoch 3/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 94s 531ms/step - accuracy: 0.5206 - loss: 1.8163 - val_accuracy: 0.4052 - val_loss: 2.3632
Epoch 4/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 97s 547ms/step - accuracy: 0.6084 - loss: 1.4650 - val_accuracy: 0.4144 - val_loss: 2.3307
Epoch 5/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 95s 539ms/step - accuracy: 0.7028 - loss: 1.1159 - val_accuracy: 0.4095 - val_loss: 2.4342
Epoch 6/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 104s 586ms/step - accuracy: 0.7871 - loss: 0.8284 - val_accuracy: 0.4173 - val_loss: 2.5160
Epoch 7/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 103s 581ms/step - accuracy: 0.8300 - loss: 0.6527 - val_accuracy: 0.4130 - val_loss: 2.4772
Epoch 8/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 104s 585ms/step - accuracy: 0.9005 - los

In [24]:
# Evaluate the model on the test set
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

C:\Users\bowma\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Test Accuracy: 40.59%


In [30]:
# Save the trained model to a file
model.save('food_recognition_model.h5')

In [32]:
# Load the saved model
loaded_model = load_model("food_recognition_model.h5")

In [33]:
# Function to predict food category from an image path
def predict_food(image_path, model):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)  # Preprocess the image

    # Predict the class 
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]  # Get the index of the highest probability

    # Get the class label based on the predicted class index
    class_labels = list(train_images.class_indices.keys())
    predicted_label = class_labels[predicted_class]

    return predicted_label


In [44]:
# Test the prediction function with a sample image
image_path = r"C:\Users\bowma\Programming\Datasets\images\lasagna\499765.jpg"
predicted_food = predict_food(image_path, loaded_model)
print(f"The predicted food is: {predicted_food}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
The predicted food is: lasagna
